In [ ]:
import json
import shutil
from pathlib import Path
from typing import Optional

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import plotly.express as px
import requests
import seaborn as sns
from IPython.display import display

from utils.preprocessing_utils import (clean_labs, clean_notas,
                                       clean_sociodemograficos,
                                       disease_tests_list)


In [ ]:
sns.set_style("whitegrid")
sns.set_context("talk")

cleaning_dict_path = "utils/lab_test_name_aggregation.json"

In [ ]:
def download_file(url: str, local_filename: Optional[str] = None):
    if local_filename is None:
        local_filename = url.split("/")[-1]
    with requests.get(url, stream=True) as r:
        with open(local_filename, "wb") as f:
            shutil.copyfileobj(r.raw, f)
    return local_filename


def unzip_file(path_to_zip_file: str, directory_to_extract_to: str):
    shutil.unpack_archive(path_to_zip_file, directory_to_extract_to)


In [ ]:
# Check if the data already exists, if not, download and unzip it
data_path = Path("data")
if (
    not (data_path / "laboratorios.csv").is_file()
    and not (data_path / "notas.csv").is_file()
    and not (data_path / "sociodemografico.csv").is_file()
):
    data_zip = download_file(
        "https://www.dropbox.com/sh/xgs3kyvyn7lmr6p/AACB4eORnqsJpRsjv9-56eUHa?dl=1",
        "data.zip",
    )
    unzip_file(data_zip, data_path)
    unzip_file(
        data_path / "Diana Buitrago - IQVIA_NLPmediaclNotes_DianaBuitrago.zip",
        data_path,
    )


In [ ]:
# Check if the data already exists, if not, download and unzip it
data_path = Path("data")
if (
    not (data_path / "laboratorios.csv").is_file()
    and not (data_path / "notas.csv").is_file()
    and not (data_path / "sociodemografico.csv").is_file()
):
    data_zip = download_file(
        "https://www.dropbox.com/sh/xgs3kyvyn7lmr6p/AACB4eORnqsJpRsjv9-56eUHa?dl=1",
        "data/data.zip",
    )
    unzip_file(data_zip, data_path)
    unzip_file(
        data_path / "Diana Buitrago - IQVIA_NLPmediaclNotes_DianaBuitrago.zip",
        data_path,
    )


## Load Data

In [ ]:
notas = pd.read_csv(str(data_path / "notas.csv"), sep=";")
notas.head()

In [ ]:
sociodemografico = pd.read_csv(str(data_path / "sociodemografico.csv"), sep=";")
sociodemografico.head()

In [ ]:
laboratorios = pd.read_csv(str(data_path / "laboratorios.csv"), sep=";")
laboratorios.head()

### Cleanup based on initial EDA

In [ ]:
with open(cleaning_dict_path, "r") as in_file:
    dict_tests = json.load(in_file)

laboratorios = clean_labs(laboratorios, name_aggregation_dict=dict_tests)

In [ ]:
notas = clean_notas(notas)
notas.head()

In [ ]:
sociodemografico = clean_sociodemograficos(sociodemografico)
sociodemografico.head()

In [ ]:
merged = sociodemografico.merge(notas, how="inner", on="IDRecord")
merged.head(5)

Merge the laboratory with the medical notes data, to get the disease tied to the Lab IDRecord. Use unduplicated data for the lab info so we don't get duplicated information

In [ ]:
merged_lab = laboratorios.merge(
    notas[["IDRecord", "Código", "Nombre"]].drop_duplicates(subset="IDRecord"),
    how="inner",
    on="IDRecord",
    suffixes=["_lab", None],
)
merged_lab.head(5)

 # Sociodemographic vs medical notes EDA

In [ ]:
# Define an auxiliary function
def percentage_for_code(df, row):
    count = row["Count"]
    total_for_code = df[df.Código == row["Código"]].Count.sum()
    return round(100 * count / total_for_code, 2)

In [ ]:
var_of_interest = "Edad"

df_analysis = (
    merged[["Nombre", "Código", var_of_interest]].value_counts().to_frame("Count")
)
fig = px.histogram(
    merged,
    x="Edad",
    color="Código",
    labels={
        "count": "Count",
    },
)
fig.show()

In [ ]:
var_of_interest = "Edad"

fig = px.box(merged, y="Edad", color="Código", notched=True)
fig.show()

In [ ]:
var_of_interest = "Genero"

df_analysis = (
    merged[["Nombre", "Código", var_of_interest]].value_counts().to_frame("Count")
)
df_analysis["Percentage"] = (df_analysis.Count / sum(df_analysis.Count) * 100).round(2)
df_analysis.sort_values(["Código", "Count"], ascending=False)

df_analysis = df_analysis.reset_index()
df_analysis["Percentage"] = df_analysis.apply(
    lambda row: percentage_for_code(df_analysis, row), axis=1
)
df_analysis = df_analysis.sort_values(["Código"])

fig = px.bar(
    df_analysis,
    x="Código",
    y=["Percentage"],
    color=var_of_interest,
    # text=df_analysis['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),
    text=df_analysis["Count"],
    labels={
        "value": "Percentage",
    },
)
fig.show()


In [ ]:
var_of_interest = "GrupoEtnico"

df_analysis = (
    merged[["Nombre", "Código", var_of_interest]].value_counts().to_frame("Count")
)
df_analysis["Percentage"] = (df_analysis.Count / sum(df_analysis.Count) * 100).round(2)
df_analysis.sort_values(["Código", "Count"], ascending=False)

df_analysis = df_analysis.reset_index()
df_analysis["Percentage"] = df_analysis.apply(
    lambda row: percentage_for_code(df_analysis, row), axis=1
)
df_analysis = df_analysis.sort_values(["Código"])

fig = px.bar(
    df_analysis,
    x="Código",
    y=["Percentage"],
    color=var_of_interest,
    # text=df_analysis['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),
    text=df_analysis["Count"],
    labels={
        "value": "Percentage",
    },
)
# fig.update_layout(height=400, width=800)
# fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()


In [ ]:
var_of_interest = "AreaResidencial"

df_analysis = (
    merged[["Nombre", "Código", var_of_interest]].value_counts().to_frame("Count")
)
df_analysis["Percentage"] = (df_analysis.Count / sum(df_analysis.Count) * 100).round(2)
df_analysis.sort_values(["Código", "Count"], ascending=False)

df_analysis = df_analysis.reset_index()
df_analysis["Percentage"] = df_analysis.apply(
    lambda row: percentage_for_code(df_analysis, row), axis=1
)
df_analysis = df_analysis.sort_values(["Código"])

fig = px.bar(
    df_analysis,
    x="Código",
    y=["Percentage"],
    color=var_of_interest,
    # text=df_analysis['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),
    text=df_analysis["Count"],
    labels={
        "value": "Percentage",
    },
)
# fig.update_layout(height=400, width=800)
# fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()


In [ ]:
var_of_interest = "EstadoCivil"

df_analysis = (
    merged[["Nombre", "Código", var_of_interest]].value_counts().to_frame("Count")
)
df_analysis["Percentage"] = (df_analysis.Count / sum(df_analysis.Count) * 100).round(2)
df_analysis.sort_values(["Código", "Count"], ascending=False)

df_analysis = df_analysis.reset_index()
df_analysis["Percentage"] = df_analysis.apply(
    lambda row: percentage_for_code(df_analysis, row), axis=1
)
df_analysis = df_analysis.sort_values(["Código"])

fig = px.bar(
    df_analysis,
    x="Código",
    y=["Percentage"],
    color=var_of_interest,
    # text=df_analysis['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),
    text=df_analysis["Count"],
    labels={
        "value": "Percentage",
    },
)
# fig.update_layout(height=400, width=800)
# fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()


In [ ]:
var_of_interest = "TSangre"

df_analysis = (
    merged[["Nombre", "Código", var_of_interest]].value_counts().to_frame("Count")
)
df_analysis["Percentage"] = (df_analysis.Count / sum(df_analysis.Count) * 100).round(2)
df_analysis.sort_values(["Código", "Count"], ascending=False)

df_analysis = df_analysis.reset_index()
df_analysis["Percentage"] = df_analysis.apply(
    lambda row: percentage_for_code(df_analysis, row), axis=1
)
df_analysis = df_analysis.sort_values(["Código"])

fig = px.bar(
    df_analysis,
    x="Código",
    y=["Percentage"],
    color=var_of_interest,
    # text=df_analysis['Percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),
    text=df_analysis["Count"],
    labels={
        "value": "Percentage",
    },
)
# fig.update_layout(height=400, width=800)
# fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()


# Target Feature Class Distribution

Let's check our taget feature distribution

In [ ]:
name = notas[["Nombre", "Código"]].value_counts().to_frame("Count")
name["Percentage"] = (name.Count / sum(name.Count) * 100).round(2)
name


After the data cleanup, we only have less than 1% for two of the 9 classes, representing a total of ~1000 samples of the ~150,000 in our dataset.
We can't easily create a prediction algorithm out of these small number of samples, so let's either drop them or merge them into similar categories.

## Classes merge

Let's try merging A510 and A511 with A514, as they all belong to the [A51 Early syphilis](https://icd.who.int/browse10/2019/en#/A51) ICD-10 denomination, indicating they share symptoms. The equivalent Spanish name for this category is [Sífilis precoz](http://ais.paho.org/classifications/chapters/CAP01.html?zoom_highlight=a51).

We could also treat A529 as part of A539, in order to reduce the number of classes without losing track of the important subcategories we already have.

In [ ]:
from utils.preprocessing_utils import merge_classes

notas = merge_classes(notas)
name = notas[["Nombre", "Código"]].value_counts().to_frame("Count")
name["Percentage"] = (name.Count / sum(name.Count) * 100).round(2)
name


# Feature Engineering

## Word count

According to the [CDC](https://www.cdc.gov/std/syphilis/stdfact-syphilis-detailed.htm), primary syphilis is characterized by a chancre mark where the disease enters the body. There is also a possibility of having extra sores in your body, but there does not seem to be any difference per se in the development of its condition based on where the disease started. Another clear indication of syphilis is Saber shin (pierna/tibia en sable).
There is also a reduction in cognitive abilities for patients who have been suffering of syphilis for some time, and this can be tested for using a simple test called the [Clock Drawing Test](https://www2.gov.bc.ca/assets/gov/health/practitioner-pro/bc-guidelines/cogimp-clock-drawing-test.pdf), or [Test del Reloj](https://www.sanitas.es/sanitas/seguros/es/particulares/biblioteca-de-salud/tercera-edad/demencias/test-reloj.html) in Spanish.

Additionally, a main characteristic of primary syphilis seems to be chancres, as well as sores for both Primary and Secondary Syphilis, making a case for creating a new numerical variable called "chancres". Another main characteristic of Syphilis is the push to use preservatives in order to reduce the possibility of other people being infected as well, which could help differentiate between Syphilis and Diabetes.

Other related keywords we can try is genital, skin (lesions), headache, HIV, serology, hepatitis, and specific tests performed on the patients.

In [ ]:
notas_eda = notas.copy()
words_to_check = [
    "chancro",
    "llaga",
    "preservativo",
    "sifili",
    "asintoma",
    "placa",
    r"(test.*reloj)",
    "sable",
    "penici",
    "antibio",
    "genital",
    "piel",
    "lesion",
    "macula",
    "cabeza",
    "vih",
    "FTA-ABS",
    "serolo",
    "hepatitis",
    "VDRL",
    "RPR",
]
aggregate_dict = {}
for word in words_to_check:
    notas_eda[word] = notas_eda.Plan.str.lower().str.count(word.lower())
    notas_eda.loc[notas_eda[word] > 1, word] = 1
    aggregate_dict[word] = ["sum"]
aggregate_dict["Nombre"] = ["count"]
notas_eda = notas_eda.groupby(["Nombre", "Código"])[
    words_to_check + ["Nombre"]
].aggregate(aggregate_dict)

for word in words_to_check:
    notas_eda.loc(axis=1)[word, "%"] = (
        notas_eda.loc(axis=1)[word, "sum"]
        / notas_eda.loc(axis=1)["Nombre", "count"]
        * 100
    ).round(2)
with pd.option_context("display.max_columns", None):
    display(notas_eda.sort_index(axis=1).sort_values(by=["Código"]))


For diabetes, we tried adding insulin and glucose as words of interest. Ketoacidosis is another relevant word which we can separate into keto and acido to see if we can capture more information.

Other keywords associated with diabetes are: obesity, carbohydrates, overweight, polyphagia, polydipsia, polyurea

In [ ]:
notas_eda = notas.copy()
words_to_check = [
    "ampolla",
    "diabet",
    "insulin",
    "gluco",
    "carbo",
    "keto",
    "acido",
    "nutri",
    "diet",
    "dependiente",
    "obes",
    "sobrepeso",
    "polifagia",
    "polidipsia",
    "poliurea",
]
aggregate_dict = {}
for word in words_to_check:
    notas_eda[word] = notas_eda.Plan.str.lower().str.count(word.lower())
    notas_eda.loc[notas_eda[word] > 1, word] = 1
    aggregate_dict[word] = ["sum"]
aggregate_dict["Nombre"] = ["count"]
notas_eda = notas_eda.groupby(["Nombre", "Código"])[
    words_to_check + ["Nombre"]
].aggregate(aggregate_dict)

for word in words_to_check:
    notas_eda.loc(axis=1)[word, "%"] = (
        notas_eda.loc(axis=1)[word, "sum"]
        / notas_eda.loc(axis=1)["Nombre", "count"]
        * 100
    ).round(2)
notas_eda.sort_index(axis=1).sort_values(by=["Código"])
with pd.option_context("display.max_columns", None):
    display(notas_eda.sort_index(axis=1).sort_values(by=["Código"]))


- There does not seem to be many mentions of chancre (chancro) or sore (llaga/placa)
- Ulcer (ulcera) does seem to be more common in patients with diabetes, although it still is negligible.
- Saber (sable) seems to be able to help differentiate for all but 1 types of syphilis against diabetes.
- asintoma seems to be useful for separating other secondary syphilis from the rest of the diseases.
- There does seem to be a significant difference between the times the word preservative (preservativo) is used between Syphilis and Diabetes.
- insulin seems like a good choice for separating E109 from the rest.
- acido, keto and diet seem to also help differentiate between syphilis and diabetes.
- Using 'diabet' and 'sifili' could help differentiate between the diagnoses of Diabetes and Syphilis.

In [ ]:
from utils.preprocessing_utils import word_count_feat_engineering

notas = word_count_feat_engineering(notas)
with pd.option_context("display.max_columns", None):
    display(notas.head(10))


## Lab results analysis

In [ ]:
merged_lab["Fecha"] = pd.to_datetime(merged_lab["Fecha"])
merged_lab


## Average and max difference in dates between each exam

In [ ]:
merged_lab_date_calc = merged_lab.sort_values(by=["IDRecord", "Fecha"]).copy()
merged_lab_date_calc["date_diff"] = (
    merged_lab_date_calc[["IDRecord", "Fecha"]].groupby("IDRecord").diff()
)
merged_lab_datediff = (
    merged_lab_date_calc[["IDRecord", "date_diff"]]
    .groupby("IDRecord")
    .agg([np.nanmean, np.nanmax])
)
merged_lab_datediff.columns = [
    "_".join(col) for col in merged_lab_datediff.columns.values
]
merged_lab_datediff = merged_lab_datediff.rename(
    columns={"date_diff_nanmean": "date_diff_mean", "date_diff_nanmax": "date_diff_max"}
)
merged_lab_datediff["date_diff_max"] = merged_lab_datediff["date_diff_max"].dt.days
merged_lab_datediff["date_diff_mean"] = merged_lab_datediff["date_diff_mean"].dt.days
merged_lab = merged_lab.drop(
    columns=["date_diff_mean", "date_diff_max"], errors="ignore"
).merge(
    merged_lab_datediff[["date_diff_mean", "date_diff_max"]].reset_index(),
    how="left",
    on="IDRecord",
)


In [ ]:
fig = px.box(
    merged_lab,
    y="date_diff_mean",
    x="Código",
    color="Código",
    # notched=True, # used notched shape
    title="Box plot of lab tests mean difference by disease code",
    #  hover_data=["day"] # add day column to hover data
)
fig.show()


In [ ]:
fig = px.box(
    merged_lab,
    y="date_diff_max",
    x="Código",
    color="Código",
    # notched=True, # used notched shape
    title="Box plot of lab tests max difference by disease code",
    #  hover_data=["day"] # add day column to hover data
)
fig.show()


In [ ]:
# merged_lab[['IDRecord', 'date_diff_max']].sort_values(by=['date_diff_max'], ascending=False)
merged_lab[merged_lab.IDRecord == 91739].sort_values(by=["fecha"])


### Disease-related tests

We can check if there are specific disease-related tests performed on the patients. E.g., as we know that HIV or a high level of lymphocites can be more common in patients with Syphilis, we can look specifically for those. Similarly, glucose tests are more common in patients with Diabetes.

In [ ]:
disease_tests = disease_tests_list()
disease_tests

In [ ]:
df_id_record = merged_lab.IDRecord.drop_duplicates().to_frame("IDRecord")
for test in disease_tests:
    df_test_count = (
        merged_lab.loc[
            merged_lab.Nombre_lab.str.contains(test[0], case=False, regex=True),
            ["IDRecord"],
        ]
        .value_counts()
        .to_frame(f"{test[1]}_count")
    )
    df_test_max = (
        merged_lab.loc[
            merged_lab.Nombre_lab.str.contains(test[0], case=False, regex=True),
            ["IDRecord", "Valor"],
        ]
        .groupby("IDRecord")
        .max()
        .reset_index()
        .rename(columns={"Valor": f"{test[1]}_max"})
    )  # .value_counts().to_frame()
    df_id_record = df_id_record.merge(df_test_count, on="IDRecord", how="left").merge(
        df_test_max, on="IDRecord", how="left"
    )
df_id_record


In [ ]:
merged = merged.drop(columns=df_id_record.drop(columns=['IDRecord']).columns, errors='ignore').merge(df_id_record, on="IDRecord", how="left")
for column in df_id_record.drop(columns=['IDRecord']).columns:
    merged[column] = merged[column].fillna(0)
merged

In [ ]:
fig = px.box(
    pd.melt(merged, id_vars=['IDRecord', 'Código'], value_vars=[f'{test[1]}_count' for test in disease_tests]),
    y='value',
    x="Código",
    color="variable",
    notched=True,  # used notched shape
    title="Box plot of lab count by lab test and disease code",
    labels={
        "value": "Count",
        "variable":"Lab test",
    },
)
fig.show()

In [ ]:
fig = px.box(
    pd.melt(merged, id_vars=['IDRecord', 'Código'], value_vars=[f'{test[1]}_max' for test in disease_tests]),
    y='value',
    x="Código",
    color="variable",
    notched=True,  # used notched shape
    title="Box plot of lab max by lab test and disease code",
    labels={
        "value": "Max",
        "variable":"Lab test",
    },
)
fig.show()

### Top lab name performed by patient

In [ ]:
merged_labs = merged_lab.groupby(["IDRecord", "Nombre_lab"])
merged_labs_agg = merged_labs.aggregate(
    {"Valor": [np.nanmean, np.nanmax], "Nombre_lab": "count"}
)
merged_labs_agg.columns = ["_".join(col) for col in merged_labs_agg.columns.values]
merged_labs_agg = merged_labs_agg.rename(
    columns={
        "Nombre_lab": "lab_count",
        "Valor_nanmean": "top_lab_avg_value",
        "Valor_nanmax": "top_lab_max_value",
    }
).reset_index()
merged_labs_agg["top_lab_avg_value"] = merged_labs_agg["top_lab_avg_value"].fillna(0)
merged_labs_agg["top_lab_max_value"] = merged_labs_agg["top_lab_max_value"].fillna(0)
merged_labs_agg

top_lab_test_by_patient = merged_labs_agg.merge(
    merged_labs_agg.loc[
        merged_labs_agg.groupby("IDRecord").Nombre_lab_count.idxmax(),
        ["IDRecord", "Nombre_lab"],
    ]
).rename(columns={"Nombre_lab": "top_lab_name", "Nombre_lab_count": "top_lab_count"})
top_lab_test_by_patient


We get then the top lab name for each patient based on the number of times that lab was prescribed to each patient, with the average and max value reading of each of those lab tests.

Merge the top lab data with the merged DF

In [ ]:
merged.drop(columns=top_lab_test_by_patient.drop(columns=['IDRecord']).columns, errors='ignore').merge(top_lab_test_by_patient, how="left", on="IDRecord")
merged["top_lab_name"] = merged.top_lab_name.fillna(0)
merged["top_lab_avg_value"] = merged.top_lab_avg_value.fillna(0)
merged["top_lab_max_value"] = merged.top_lab_max_value.fillna(0)
merged["top_lab_count"] = merged.top_lab_count.fillna("NA")
merged

### Number of total lab tests taken by patient

In [ ]:
total_lab_count_by_patient = (
    merged_labs_agg.groupby(["IDRecord"])
    .aggregate({"Nombre_lab_count": "sum"})
    .rename(columns={"Nombre_lab_count": "total_Nombre_lab_count"})
)
total_lab_count_by_patient


Let's check if there's any relationship between number of tests and the disease class

In [ ]:
if "total_Nombre_lab_count" not in merged.columns:
    merged = merged.merge(total_lab_count_by_patient, how="left", on="IDRecord")
else:
    merged = merged.drop(columns="total_lab_count").merge(
        total_lab_count_by_patient, how="left", on="IDRecord"
    )
merged["total_lab_count"] = merged.total_Nombre_lab_count.fillna(0)
merged


In [ ]:
merged[["IDRecord", "total_Nombre_lab_count", "Código"]].sort_values(
    by="total_Nombre_lab_count"
)


In [ ]:
fig = px.box(
    merged,
    y="total_Nombre_lab_count",
    x="Código",
    color="Código",
    notched=True,  # used notched shape
    title="Box plot of lab count by disease code",
    #  hover_data=["day"] # add day column to hover data
)
fig.show()
